In [58]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer

In [91]:
scored_train_data = pd.read_csv('preped_train_data_wth_dummies.csv', index_col=0)
all_sales = pd.read_csv('sales_jan23_avr23.csv')
scaler = pickle.load(open('scaler_numerical.sav', 'rb'))

In [92]:
def adding_log(train: pd.DataFrame, columns_to_log: list):
    """ 
    Adding columns using log of provided information.
    """
    new_train = train.__deepcopy__()
    for column_name in columns_to_log:
        new_position = train.columns.get_loc(str(column_name)) + 1
        new_name = str ('log_' + str(column_name))
        new_train.insert(new_position, new_name, np.log(new_train[str(column_name)] + 1))
        new_train = new_train.drop(str(column_name), axis='columns')
        
    return (new_train)
    

In [94]:
def scale(df, scaler, numerical_columns: list):
    """
    Returns dataframe with the given columns scaled.
    """
    new_train = df.__deepcopy__()
    num_train_data = new_train[numerical_columns]
    new_train[numerical_columns] = scaler.transform(num_train_data)

    return (new_train)


In [95]:
def sales_pre_pocessing(all_sales, cols_to_log, scaler, cols_to_scale):

    dummies_category = pd.get_dummies(all_sales['Category'])
    dummies_badges = all_sales['Badges'].str.strip('{}').str.replace('"', '').str.get_dummies(',')
    dropped_cols_sales = all_sales.drop(['Category','Badges'], axis = 1)

    all_sales_dummies = pd.concat([dropped_cols_sales, dummies_category, dummies_badges], axis = 1)

    all_sales_logged = adding_log(all_sales_dummies, cols_to_log)

    all_sales_scaled = scale(all_sales_logged, scaler, cols_to_scale,)
    
    return all_sales_scaled

In [93]:
num_columns = ['log_Followers',
               'log_Avg Price',
               'log_First Day Revenue',
               'log_Brand Appearance',
               'Avg Discount',
               'Conversion']

In [67]:
user_id = 'zzWUNSYQ2CWwopiILtUU4kaBEm13'
user_information = scored_train_data[['User Key', 'Recency', 'Frequency', 'Monetary']]
available_dates = ['2023-03-26', '2023-04-18', '2023-04-08']
available_sales = all_sales_dummies[all_sales_dummies['Start At'].isin(available_dates)]
model = pickle.load(open('reg_model.pkl', 'rb'))

In [68]:
def creating_entry_data(user_id, user_information, available_sales: pd.DataFrame):
    """ 
    Aims at returning ranked sales for a given user
    """

    user_specific_information = user_information[user_information['User Key'] == user_id].drop('User Key', axis = 1)


    user_features = user_specific_information.columns

    df_for_predictions = available_sales.__deepcopy__().reset_index(drop = True)
    user_specific_df = pd.concat([user_specific_information] *len (df_for_predictions), ignore_index=True)

    df_for_predictions[user_features] = user_specific_df

    return df_for_predictions


In [69]:
def ranking(df_for_prediction: pd.DataFrame, model):

    predicted = df_for_prediction['Sale ID']
    predicted['score'] = 0

    df_for_prediction = df_for_prediction.drop(columns=['Sale ID', 'Start At'])

    for i in df_for_prediction.index:
        predicted_score = model.predict(df_for_prediction.loc[i])
        predicted['score'][i] = predicted_score
    
    return predicted

In [70]:
df_for_pred = creating_entry_data('yQHcG2KlpVg6TKKY0IOMI1JqDqo1',user_information, available_sales)

In [71]:
ranking(df_for_pred, model)

/var/folders/r8/txzynl1j0xj6xzvf7622s2vw0000gn/T/ipykernel_25166/3655031587.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted['score'] = 0
/Users/priscilleerulin/.pyenv/versions/3.9.16/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[7.92400000e+03 2.00000000e+00 2.54263103e+04 2.04000000e+01
 3.42013200e+03 3.31168831e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000

In [84]:
model.predict(df_for_pred.drop(columns=['Sale ID', 'Start At']).iloc[:1])

/Users/priscilleerulin/.pyenv/versions/3.9.16/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Avg Price
- Brand Appearance
- Chaussant
- Excellent sur Yuka
- First Day Revenue
- ...
Feature names seen at fit time, yet now missing:
- log_Avg Price
- log_Brand Appearance
- log_First Day Revenue
- log_Followers

  warnings.warn(message, FutureWarning)


ValueError: X has 65 features, but LinearRegression is expecting 58 features as input.

In [83]:
print(df_for_pred.drop(columns=['Sale ID', 'Start At']).iloc[1:2])

   Followers  Brand Appearance    Avg Price  Avg Discount  First Day Revenue  \
1      12883                 2  4860.614605     30.347105             725.52   

   Conversion  Artisanal  B-Corporation  Bien-être  Bio  ...  Lingerie  \
1    0.001178          0              0          0    0  ...         1   

   Maroquinerie  Outdoor  Prêt-à-porter  Salon  Soins  Sportswear  Recency  \
1             0        0              0      0      0           0        1   

   Frequency  Monetary  
1          3     129.0  

[1 rows x 65 columns]
